# [SE_Xception]

*KU LeeDongGyu*

## Contents
---

1. Data Preprocessing
```
1) Data Import
2) Data Augmentation
```
2. Support Functions & Almost Original Xception
```
1) Support Functions
2) Almost Original Xception
3) Xception Evaluate
```



### Install Packages


### Module

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install keras==2.3.0

     |████████████████████████████████| 378kB 6.1MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 36kB/s 
     |████████████████████████████████| 3.8MB 51.0MB/s 
     |████████████████████████████████| 450kB 48.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=61945c2be397aa9fbdff7eb9a628080530ed22fa12720f97e5bf1e5647be5f9b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorbo

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/DACON

/content/drive/My Drive/Colab Notebooks/DACON


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow import keras as ks
from tensorflow.keras import backend as K 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Concatenate, ZeroPadding2D ,GlobalMaxPooling2D, Reshape , Lambda , Add, Multiply
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, AveragePooling2D , ZeroPadding2D, SeparableConv2D
from tensorflow.keras.layers import add
from tensorflow.keras.optimizers import Adam, RMSprop , SGD
from tensorflow.keras.callbacks import EarlyStopping , LearningRateScheduler, ModelCheckpoint, CSVLogger, Callback, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1,l2,l1_l2
from tensorflow.keras.models import Model , load_model , Sequential
from tensorflow.keras.utils import plot_model , to_categorical, get_file
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/DACON'

In [ ]:
print(tf.__version__)
print(ks.__version__)

2.0.0
2.2.4-tf


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2428730506182394031
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 428996885344536677
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9550298734162006649
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15753943450
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5296544721345415304
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
]


## 1. Data Preprocessing
---

In [ ]:
# flow 기준

In [ ]:
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('MONTH7_MNIST/data/train.csv')

In [ ]:
zoom_train = train.drop(['id', 'digit', 'letter'], axis=1).values
zoom_train = zoom_train.reshape(-1, 28, 28, 1)
zoom_train = zoom_train/255
# tr_center = np.mean(x_train, axis=(0,1,2))

y = train['digit']
y_train = np.zeros((len(y), len(y.unique())))
for i, digit in enumerate(y):
    y_train[i, digit] = 1

In [ ]:
size = 299

In [ ]:
x_train = []
for i in range(len(zoom_train)):
    x_train.append(cv2.resize(zoom_train[i], (size,size), interpolation=cv2.INTER_CUBIC))
x_train = np.array(x_train)

In [ ]:
x_train=x_train.reshape(-1,size,size,1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=200814)

### 1) Data Import

In [ ]:
# 바꿔서 살펴 볼 것들
# CALTECH, CIFAR100, FER, MIT
super_size = 330
size = 299
input_sizes = (size,size,1)
batch_sizes = 32
classes = 10
tr_center = [0.14296818]
epochs = 500
weight_decay = 6e-5

In [ ]:
from numpy.random import seed
import random

# setting the seed number for random number generation for reproducibility.

seed_num = 200819
os.environ['PYTHONHASHSEED']=str(seed_num)
random.seed(seed_num)
seed(seed_num)
tf.random.set_seed(seed_num)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
dir = os.getcwd()

### 2) Data Augmentation

In [ ]:
datagen_tr = ImageDataGenerator(
    #rescale=1/255.,
    horizontal_flip=False,
    featurewise_center=True,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=[0.9,1.0],
    fill_mode = 'nearest')
datagen_val = ImageDataGenerator(#rescale=1/255.,
                                 featurewise_center=True)
datagen_tes = ImageDataGenerator(#rescale=1/255.,
                                 featurewise_center=True)

# 원래는 이 자리에 fit 매서드를 써야하지만, 그냥 내가 중심화함수를 만들고 적용함. 

# 중심화 설정
datagen_tr.mean = np.array(tr_center, dtype=np.float32).reshape((1,1,1)) 
datagen_val.mean = np.array(tr_center, dtype=np.float32).reshape((1,1,1))
datagen_tes.mean = np.array(tr_center, dtype=np.float32).reshape((1,1,1)) 

In [ ]:
train_generator = datagen_tr.flow(x_train,y_train, batch_size=batch_sizes)
valid_generator = datagen_val.flow(x_valid,y_valid, batch_size=batch_sizes)
#test_generator = datagen_tes.flow(x_test,y_test,batch_size= 1 )

## 2. Support Functions & Almost Original Xception
---

### 1) Support Functions

In [ ]:
# def lr_schedule(epoch):
#     init_lr = 1e-2
#     k = 0.04
#     lr = init_lr * np.exp(-k*epoch)
#     print('Learning rate: ', lr)
#     return lr

def lr_schedule(epoch):
    lr = 1e-3
    if epoch < 30:
        lr = lr
    elif epoch < 60:
        lr = lr * 0.1
    elif epoch < 90:
        lr = lr * 0.01
    print('Learning rate: ', lr)
    return lr

In [ ]:
# SE block을 만들고 붙이자.
def SE_block(input_tensor, reduction_ratio):
    ch_input = K.int_shape(input_tensor)[-1]
    ch_reduced = ch_input//reduction_ratio
    
    # Squeeze
    x = GlobalAveragePooling2D()(input_tensor) 
    
    # Excitation
    x = Dense(ch_reduced, activation='relu', kernel_initializer='he_normal', use_bias=False)(x) 
    x = Dense(ch_input, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(x) 
    
    x = Reshape( (1, 1, ch_input) )(x)
    x = Multiply()([input_tensor, x]) # broadcasting 
    
    return x

In [ ]:
def conv2d_bn(x, filters, kernel_size, weight_decay, padding='same', strides=1, activation='relu'):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, kernel_initializer='he_normal', kernel_regularizer=l1_l2(l1=0, l2=weight_decay))(x)
    x = BatchNormalization()(x)
    
    if activation:
        x = Activation(activation)(x)
    
    return x

In [ ]:
def sepconv2d_bn(x, filters, kernel_size, weight_decay, padding='same', strides=1, activation='relu', depth_multiplier=1):
    x = SeparableConv2D(filters, kernel_size, padding=padding, strides=strides, depth_multiplier=depth_multiplier, 
                        depthwise_initializer='he_normal', pointwise_initializer='he_normal',
                        depthwise_regularizer=l1_l2(l1=0, l2=weight_decay), pointwise_regularizer=l1_l2(l1=0, l2=weight_decay))(x)
    x = BatchNormalization()(x)
    
    if activation:
        x = Activation(activation)(x)
    
    return x

### 2) Almost Original SE-Xception


In [ ]:
def SE_Xception(model_input, classes, weight_decay, reduction_ratio, name = "SE_Xception"):
    ## Entry flow
    x = conv2d_bn(model_input, 32, (3, 3), weight_decay, strides=2) # (299, 299, 3) -> (150, 150, 32)
    x = conv2d_bn(x, 64, (3, 3), weight_decay)

    for fliters in [128, 256, 728]: # (75, 75, 64) -> (75, 75, 128) -> (38, 38, 256) -> (19, 19, 728)
        residual = conv2d_bn(x, fliters, (1, 1), weight_decay, strides=2, activation=None)
        
        x = Activation(activation='relu')(x)
        x = sepconv2d_bn(x, fliters, (3, 3), weight_decay)
        x = sepconv2d_bn(x, fliters, (3, 3), weight_decay, activation=None)
        x = MaxPooling2D((3, 3), padding='same', strides=2)(x)

        # SE block 추가
        x =  SE_block(x, reduction_ratio)
    
        x = Add()([x, residual])
        
        
    ## Middle flow
    for i in range(8): # (19, 19, 728)
        residual = x
        
        x = Activation(activation='relu')(x)
        x = sepconv2d_bn(x, 728, (3, 3), weight_decay)
        x = sepconv2d_bn(x, 728, (3, 3), weight_decay)
        x = sepconv2d_bn(x, 728, (3, 3), weight_decay, activation=None)

        # SE block 추가
        x =  SE_block(x,reduction_ratio)
        
        x = Add()([x, residual])
        
        
    ## Exit flow
    residual = conv2d_bn(x, 1024, (1, 1), weight_decay, strides=2, activation=None) # (19, 19, 728) -> (10, 10, 1024)
        
    x = Activation(activation='relu')(x)
    x = sepconv2d_bn(x, 728, (3, 3), weight_decay)
    x = sepconv2d_bn(x, 1024, (3, 3), weight_decay, activation=None) # (19, 19, 728) -> (19, 19, 1024)
    x = MaxPooling2D((3, 3), padding='same', strides=2)(x) # (19, 19, 1024) -> (10, 10, 1024)

    # SE block 추가
    x =  SE_block(x, reduction_ratio)
    
    x = Add()([x, residual])
    
    x = sepconv2d_bn(x, 1536, (3, 3), weight_decay)
    x = sepconv2d_bn(x, 2048, (3, 3), weight_decay)

    x = GlobalAveragePooling2D()(x)
    
    ## Optinal fully-connected layers
    '''
    x = Dense(4096)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    x = Dense(4096)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    '''
    
    x = Dropout(0.5)(x)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x)

    model = Model(model_input, model_output, name=name)
    
    return model

In [ ]:
model_input = Input( shape=input_sizes )
model = SE_Xception(model_input, classes=classes, weight_decay = weight_decay, reduction_ratio = 16, name='test6')

In [ ]:
model.summary()

Model: "test6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 150, 150, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 150, 150, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 150, 150, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
# 폴더 생성

os.makedirs(os.path.join(dir,'MONTH7_MNIST/model_output',model.name), exist_ok=True)
os.makedirs(os.path.join(dir,'MONTH7_MNIST/train_valid_output'), exist_ok=True)

In [ ]:
# adamwr 시작
# 출처 : https://github.com/OverLordGoldDragon/keras-adamw
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Paper')
import utils
import optimizers_v2
from utils import get_weight_decays, fill_dict_in_order
from utils import reset_seeds, K_eval
from optimizers_v2 import AdamW, NadamW, SGDW

In [ ]:
optimizer = AdamW(model=model, use_cosine_annealing=True, total_iterations = len(x_train) // batch_sizes )
#optimizer = tf.keras.optimizers.Adam()

NameError: ignored

In [ ]:
filepath =  os.path.join(dir,'MONTH7_MNIST/model_output',model.name,'{epoch:03d}.h5')

callbacks_list = [ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=False, save_best_only=True, mode='min'),
                  ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_weights_only=False, save_best_only=True, mode='max')
                  #ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.1,min_lr=1e-5),
                  #LearningRateScheduler(lr_schedule,verbose=1)
                  ]
                  
model.compile(optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
######## flow_from_directory
history = model.fit(train_generator, steps_per_epoch=int(len(x_train)/batch_sizes),  validation_data = valid_generator, epochs=epochs , verbose=1 , callbacks = callbacks_list , validation_steps=int(len(x_valid)/batch_sizes))

Train for 51 steps, validate for 12 steps
Epoch 1/500
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_1/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_2/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_3/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_4/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_5/kernel:0
50/51 [============================>.] - ETA: 1s - loss: 4.9564 - accuracy: 0.2751
Epoch 00001: val_loss improved from inf to 5.19936, saving model to /content/drive/My Drive/Colab Notebooks/DACON/MONTH7_MNIST/model_output/test6/001.h5

Epoch 00001: val_accuracy improved from -inf to 0.11458, saving model to /content/drive/My Drive/Colab Notebooks/DACON/MONTH7_MNIST/model_output/test6/001.h5
51/51 [==============================] - 87s 2s/step - loss: 4.9502 - accuracy: 0.2771 - val_loss: 5.1994 - v

In [ ]:
# model fit

In [ ]:
model=load_model(os.path.join(dir,'MONTH7_MNIST/model_output/test6/398.h5'),custom_objects={'AdamW':AdamW})

Using cosine annealing learning rates
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_1/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_2/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_3/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_4/kernel:0
0.0(L1), 8.401680291923756e-06(L2) weight decay set for conv2d_5/kernel:0


In [ ]:
test = pd.read_csv('MONTH7_MNIST/data/test.csv')
size = 299
zoom_test = test.drop(['id', 'letter'], axis=1).values
zoom_test = zoom_test.reshape(-1,28,28,1) / 255
zoom_test = zoom_test - tr_center

In [ ]:
x_test = []

for i in range(len(zoom_test)//4):
    x_test.append(cv2.resize(zoom_test[i], (size,size), interpolation=cv2.INTER_CUBIC))
x_test = np.array(x_test)
x_test = x_test.reshape(-1, size, size, 1)
temp1 = np.argmax(model.predict(x_test), axis=1)

In [ ]:
x_test = []

for i in range(len(zoom_test)//4,len(zoom_test)//4 * 2):
    x_test.append(cv2.resize(zoom_test[i], (size,size), interpolation=cv2.INTER_CUBIC))
x_test = np.array(x_test)
x_test = x_test.reshape(-1, size, size, 1)
temp2 = np.argmax(model.predict(x_test), axis=1)

In [ ]:
x_test = []

for i in range(len(zoom_test)//4 * 2,len(zoom_test)//4 *3):
    x_test.append(cv2.resize(zoom_test[i], (size,size), interpolation=cv2.INTER_CUBIC))
x_test = np.array(x_test)
x_test = x_test.reshape(-1, size, size, 1)
temp3 = np.argmax(model.predict(x_test), axis=1)

In [ ]:
x_test = []

for i in range(len(zoom_test)//4 * 3,len(zoom_test)):
    x_test.append(cv2.resize(zoom_test[i], (size,size), interpolation=cv2.INTER_CUBIC))
x_test = np.array(x_test)
x_test = x_test.reshape(-1, size, size, 1)
temp4 = np.argmax(model.predict(x_test), axis=1)

In [ ]:
final = np.concatenate([temp1,temp2,temp3,temp4])

In [ ]:
submission = pd.read_csv('MONTH7_MNIST/data/submission.csv')
submission['digit'] = final
submission.head()

,id,digit
0,2049,6
1,2050,9
2,2051,8
3,2052,0
4,2053,3


In [ ]:
submission.to_csv('MONTH7_MNIST/data/se_xception_adamwr.csv', index=False)

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
epochs=range(1,len(acc)+1)

NameError: ignored

In [ ]:
plt.plot(epochs[1:],acc[1:],'b',label='Training Acc')
plt.plot(epochs[1:],val_acc[1:],'r',label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()
plt.plot(epochs[10:],loss[10:],'b',label='Training Loss')
plt.plot(epochs[10:],val_loss[10:],'r',label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()